# Goal Recognition as a Deep Learning Task: the GRNet Approach

## Imports 

In [145]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
import tensorflow.keras.backend as K
import time
import os

from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras.initializers import Constant
from keras.losses import BinaryCrossentropy
from os.path import join
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import load_model
from tqdm import tqdm
from typing import Dict, List, Union

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Custom Classes

### Network classes

Code from 
*Yang, Z.; Yang, D.; Dyer, C.; He, X.; Smola, A. J.; and Hovy, E. H.* 2016. **Hierarchical Attention Networks for Document Classification**
https://github.com/philipperemy/keras-attention-mechanism

In [146]:
class AttentionWeights(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
        # self.init = initializers.get(Constant(value=1))

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(AttentionWeights, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        return a

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

    def get_config(self):
        config={'step_dim':self.step_dim}
        base_config = super(AttentionWeights, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    




class ContextVector(Layer):
    def __init__(self, **kwargs):
        super(ContextVector, self).__init__(**kwargs)
        self.features_dim = 0

    def build(self, input_shape):
        assert len(input_shape) == 2
        self.features_dim = input_shape[0][-1]
        self.built = True

    def call(self, x, **kwargs):
        assert len(x) == 2
        h = x[0]
        a = x[1]
        a = K.expand_dims(a)
        weighted_input = h * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0][0], self.features_dim

    def get_config(self):
        base_config = super(ContextVector, self).get_config()
        return dict(list(base_config.items()))

### Constants class

In [147]:
class C:
    '''
    Constants class.
    '''
    OBSERVATIONS = 0
    CORRECT_GOAL = 1
    POSSIBLE_GOALS = 2 
    
    SATELLITE = 0
    LOGISTICS = 1
    ZENOTRAVEL = 2
    BLOCKSWORLD = 3
    DRIVERLOG = 4
    DEPOTS = 5
    
    MAX_PLAN_LENGTH = 0
    MODEL_FILE = 1
    DICTIONARIES_DICT = 2
    
    SMALL = 0
    COMPLETE = 1
    PERCENTAGE = 2
    
    MODELS_DIR = '../models/'
    DICTIONARIES_DIR = './dictionaries/'
    #MODELS_DIR = './incremental_models/'
    
    MODEL_LOGISTICS = None
    MODEL_SATELLITE = None
    MODEL_ZENOTRAVEL = None
    MODEL_BLOCKSWORLS = None
    MODEL_DRIVERLOG = None
    MODEL_DEPOTS = None

    MAX_PLAN_PERCENTAGE = 0.7

    TABLE_HEADERS = ['', 'Pereira', 'Our', 'Support']
    
    CUSTOM_OBJECTS = {'AttentionWeights': AttentionWeights,
                   'ContextVector' : ContextVector,
                   'custom_multilabel_loss_v3' : BinaryCrossentropy}


### Exceptions

In [148]:
class PlanLengthError(Exception):
    pass

class FileFormatError(Exception):
    pass

class UnknownIndexError(Exception):
    pass

## Custom Methods

### Unpack files methods

In [149]:
def unzip_file(file_path: str, target_dir: str) -> None:
    '''
    Unzip a file in an empty directory. The directory is 
    emptied before the execution.
    
    Args:
        file_path:
            A string that contains the path
            to the .zip file.
        
        target_dir:
            A string that contains the path 
            to the target directory. This 
            directory is created if it doesn't
            exist and it is emptied if it exists.
        
    '''
    if os.path.exists(target_dir):
        for f in os.listdir(target_dir):
            os.remove(join(target_dir, f))
        os.rmdir(target_dir)
    os.mkdir(target_dir)
    os.system(f'unzip -qq {file_path} -d {target_dir}')
    
def unpack_bz2(file_path: str, target_dir: str) -> None:
    '''
    Unpack a .bz2 file in an empty directory. The directory 
    is emptied before the execution.
    
    Args:
        file_path:
            A string that contains the path
            to the .bz2 file.
        
        target_dir:
            A string that contains the path 
            to the target directory. This 
            directory is created if it doesn't
            exist and it is emptied if it exists.
        
    '''
    if os.path.exists(target_dir):
        for f in os.listdir(target_dir):
            os.remove(join(target_dir, f))
        os.rmdir(target_dir)
    os.mkdir(target_dir)
    os.system(f'tar -xf {file_path} -C {target_dir}')

### Input parse methods

In [150]:
def load_file(file: str, binary: bool = False, use_pickle: bool = False):
    '''
    Get file content from path.
    
    Args:
        file:
            A string that contains the path
            to the file.
        binary:
            Optional. True if the file is a 
            binary file.
        use_pickle:
            Optional. True if the file was 
            saved using pickle.
            
    Returns:
        The content of the file.
    
    Raises:
        FileNotFoundError:
            An error accessing the file
    '''
    operation = 'r'
    if binary:
        operation += 'b'
    with open(file, operation) as rf:
        if use_pickle:
            output = pickle.load(rf)
        else:
            output = rf.readlines()
        rf.close()
    return output
        
        

In [151]:
def parse_file(read_file: str, content_type: int, dictionary: dict = None):
    '''
    Parse different input files.
    
    Args:
        read_file: 
            String containing the path to the file.
        content_type: 
            Integer representing the kind of parse to apply.
                0: observations file,
                1: correct goal file, 
                2: possible goals file
        
    Returns:
        A list of strings that contains the parsed elements.
        
    Raises:
        FileFormatError: 
            An error regarding the action format in 
            the file   
    '''
    
    msg_empty = f'File {read_file} is empty.'
    msg_index = f'Content type {content_type} is unknown.' 
    
    elements = list()
    
    lines = load_file(read_file)
    if len(lines) == 0:
        raise FileFormatError(msg_empty)
    if content_type == C.OBSERVATIONS:
        elements = parse_observations(lines, dictionary)
    elif content_type == C.POSSIBLE_GOALS:
        elements = parse_possible_goals(lines, dictionary)
    elif content_type == C.CORRECT_GOAL:
        elements = parse_correct_goal(lines[0], dictionary)
    else:
        raise UnknownIndexError(msg_index)
    
    if len(elements) > 0:    
        return elements
    else:
        raise FileFormatError(msg_empty)
        

def remove_parentheses(line: str) -> str:
    '''
    Remove parentheses from a string.
    
    Args:
        line: a string that is enclosed in parentheses.
        For example:
        
        "(string example)"
        
    Returns:
        The string without the parenteses.
        None if the string is empty.
        
    Raises:
        FileFormatError: error handling the string
    '''
    
    msg = (f'Error while parsing a line. Expected "(custom '
    +f'text)" but found "{line}"')
    
    line = line.strip()
    if line.startswith('(') and line.endswith(')'):
        element = line[1:-1]
        element = element.strip()
        if len(element) == 0:
            return None
        else:
            return element
    elif len(line) == 0:
        return None
    else:
        raise FileFormatError(msg)
        
def retrieve_from_dict(key: str, dictionary: dict):
    '''
    Return the dictionary value given the key.
    
    Args:
        key:
            A string that is the key.
        dictionary:
            A dict.
            
    Returns:
        The value corresponding to the key.
    
    Raises:
        KeyError:
            An error accessing the dictionary.
    '''
    
    msg_error = f'Key {key.upper()} is not in the dictionary'
    
    try:
        return dictionary[key.upper()]
    except KeyError:
        print(msg_error)
        np.random.seed(47)
        return np.random.randint(0,len(dictionary))

def parse_correct_goal(line: str, goals_dict: dict = None) -> list:
    '''
    Parse the fluents that compose a goal.
    
    Args:
        line: 
            A string that contains one or more 
            fluents in the goal. Fluents are 
            enclosed in parentheses and separated
            by commas. For example:
            
            "(fluent1), (fluent2),  (fluent3)"
        
        goals_dict:
            Optional. A dictionary that maps each 
            fluent to its unique identifier.
    
    Returns:
        A list of strings containing each fluent 
        without parentheses.
        
    Raises:
        FileFormatError:
            An error accessing the file.
    '''
    msg_empty = 'Parsed goal is empty.'
    
    goal = list()
    line = line.strip()
    fluents = line.split(',')
    for f in fluents:
        fluent = remove_parentheses(f)
        if fluent is not None:
            if goals_dict is not None:
                fluent = retrieve_from_dict(fluent, goals_dict)
            goal.append(fluent)
    if len(goal) > 0:
        return goal
    else:
        raise FileFormatError(msg_empty)
    

        
def parse_observations(lines: list, obs_dict: dict = None) -> list:
    '''
    Removes parentheses and empty strings from 
    the observations list.
    
    Args:
        lines: 
            List of strings that contains the 
            observations. Each observation is
            enclosed in parentheses. For 
            example:
            
            ['(observation1)', '', '(observation2)']
        
        obs_dict:
            Optional. A dictionary that maps each 
            observation to its unique identifier.
            
    Returns:
        The input list without parentheses and
        empty strings.
        
    Raises:
        FileFormatError:
            An error accessing the file.
    '''
    msg_empty='Observations list is empty.'
    
    observations = list()
    
    for line in lines:
        observation = remove_parentheses(line)
        if observation is not None:
            if obs_dict is not None:
                observation = retrieve_from_dict(observation, obs_dict)
            observations.append(observation)
    if len(observations)>0:
        return observations
    else:
        raise FileFormatError(msg_empty)

def parse_possible_goals(lines: list, goals_dict: dict = None) -> list:
    '''
    Parse a list of goals.
    
    Args:
        lines:
            A list of strings that contains each
            possible goal.
            
        goals_dict:
            Optional. A dictionary that maps each 
            fluent to its unique identifier.
    
    Returns:
        A list of lists. Each list contains the fluents
        that compose the goal represented as a string.
        
    Raises:
        FileFormatError:
            An error accessing the file.
    '''
    msg_empty='Possible goals list is empty.'
    
    goals=list()
    for line in lines:
        line = line.strip()
        if len(line)>0:
            goals.append(parse_correct_goal(line, goals_dict))
    if len(goals) > 0:
        return goals
    else:
        raise FileFormatError(msg_empty)
            
            

### Model related methods

In [152]:
def parse_domain(domain: Union[str, int]) -> int:
    '''
    Converts domain name into integer
    
    Args:
        domain: 
            A string or an int that represents
            a domain.
    
    Returns:
        An integer associated to a specific domain.
        
    Raises:
        KeyError:
            An error parsing the domain arg.
    '''
    msg = (f'Provided domain {domain} is not supported. '+
           f'Supported domains are: {C.SATELLITE} : satellite, ' +
           f'{C.LOGISTICS} : logistics, {C.BLOCKSWORLD} : blocksworld, ' +
           f'{C.ZENOTRAVEL} : zenotravel, {C.DRIVERLOG}: driverlog,' + 
           f'{C.DEPOTS}: depots.')
           
    if (str(domain).isdigit() and int(domain) == C.SATELLITE) or str(domain).lower().strip() == 'satellite':
        return C.SATELLITE
    elif (str(domain).isdigit() and int(domain) == C.LOGISTICS) or str(domain).lower().strip() == 'logistics':
        return C.LOGISTICS
    elif (str(domain).isdigit() and int(domain) == C.BLOCKSWORLD) or str(domain).lower().strip() == 'blocksworld':
        return C.BLOCKSWORLD
    elif (str(domain).isdigit() and int(domain) == C.ZENOTRAVEL) or str(domain).lower().strip() == 'zenotravel':
        return C.ZENOTRAVEL
    elif (str(domain).isdigit() and int(domain) == C.DRIVERLOG) or str(domain).lower().strip() == 'driverlog':
        return C.DRIVERLOG
    elif (str(domain).isdigit() and int(domain) == C.DEPOTS) or str(domain).lower().strip() == 'depots':
        return C.DEPOTS
    else:
        raise KeyError(msg)

In [153]:
def get_model(domain: int):
    '''
    Loads the model for a specific domain.
    
    Args:
        domain: 
            an integer associated to a specific 
            domain.
            
    Returns:
        The Model loaded for the domain or None
        if there is no model in memory.
        
    Raises:
        KeyError:
            An error parsing the domain arg.
    '''

    msg = (f'Provided domain {domain} is not supported. '+
       f'Supported domains are: {C.SATELLITE} : satellite, ' +
       f'{C.LOGISTICS} : logistics, {C.BLOCKSWORLD} : blocksworld, ' +
       f'{C.ZENOTRAVEL} : zenotravel, {C.DRIVERLOG}: driverlog,' + 
       f'{C.DEPOTS}: depots.')
    
    if domain == C.LOGISTICS:
        return C.MODEL_LOGISTICS
    elif domain == C.SATELLITE:
        return C.MODEL_SATELLITE
    elif domain == C.DEPOTS:
        return C.MODEL_DEPOTS
    elif domain == C.BLOCKSWORLD:
        return C.MODEL_BLOCKSWORLS
    elif domain == C.DRIVERLOG:
        return C.MODEL_DRIVERLOG
    elif domain == C.ZENOTRAVEL:
        return C.MODEL_ZENOTRAVEL
    else:
        raise KeyError(msg)

In [154]:
def get_domain_related(domain: int, element: int, model_type: int = C.SMALL, 
                       percentage: float = 0) -> Union[int, str]:
    '''
    Returns domain related information
    
    Args:
        domain: 
            an integer associated to a specific 
            domain.
        
        element:
            an integer associated to a specific
            piece of information to retrieve.
        
        model_type:
            an integer associated to the type
            of RNN model in use.
        
        percentage:
            a float that represents the model
            percentage to use. Use only with
            model_type = C.PERCENTAGE.
    
    Returns: 
        Max plan size if element=C.MAX_PLAN_LENGTH,
        Model file if element=C.MODEL_FILE
        Dictionaries directory if element=C.DICTIONARIES_DICT
        
    '''
    
    msg = (f'Provided domain {domain} is not supported. '+
           f'Supported domains are: {C.SATELLITE} : satellite, ' +
           f'{C.LOGISTICS} : logistics, {C.BLOCKSWORLD} : blocksworld, ' +
           f'{C.ZENOTRAVEL} : zenotravel.')
    if domain == C.LOGISTICS:
        v = {
            'max_plan_len' : 50,
            'name' : 'logistics',
        }
    elif domain == C.SATELLITE:
        v = {
            'max_plan_len' : 40,
            'name' : 'satellite',
        }
    elif domain == C.ZENOTRAVEL:
        v = {
            'max_plan_len' : 40,
            'name' : 'zenotravel',
        }
    elif domain == C.BLOCKSWORLD:
        v = {
            'max_plan_len' : 75,
            'name' : 'blocksworld',
        }
    elif domain == C.DRIVERLOG:
        v = {
            'max_plan_len' : 70,
            'name' : 'driverlog',
        }
    elif domain == C.DEPOTS:
        v = {
            'max_plan_len' : 64,
            'name' : 'depots'
        }
    else:
        raise KeyError(msg)
        
    if element == C.MAX_PLAN_LENGTH:
        return int(v['max_plan_len']*C.MAX_PLAN_PERCENTAGE)
    
    elif element == C.MODEL_FILE:
        if model_type == C.COMPLETE:
            return f'{v["name"]}.h5'
        elif model_type == C.SMALL:
            return f'{v["name"]}_small.h5'
        elif model_type == C.PERCENTAGE:
            return f'{v["name"]}_{int(percentage*100)}perc.h5'
        
    elif element == C.DICTIONARIES_DICT:
        return join(C.DICTIONARIES_DIR, f'{v["name"]}')


### Domain component methods

In [155]:
def get_observations_array(observations: list, max_plan_length: int) -> np.ndarray:
    '''
    Create an array of observations index.
    
    Args:
        observations: 
            A list of action names
            
        max_plan_length:
            An integer that contains the maximum size of
            the list that will be considered.
    
    Returns:
        An array that contains the observations' indexes
    '''
    
    WARNING_MSG = (f'The action trace is too long. Only the first {max_plan_length}'+
                 f'actions will be considered.')
    
    observations_array = np.zeros((1, max_plan_length))
    if len(observations) > max_plan_length:
        pass
        # print(WARNING_MSG)
    for index, observation in enumerate(observations):
        if index < max_plan_length:
            observations_array[0][index] = int(observation)
    return observations_array
        

def get_predictions(observations: list, 
                    max_plan_length: int, 
                    domain: int) -> np.ndarray:
    '''
    Return the model predictions.
    
    Args:
        observations:
            A list of action names.
        
        max_plan_length:
            An integer that contains the maximum size of
            the list that will be considered.
        
        domain:
            An integer associated to a specific domain.
    
    Returns:
        The model predictions.
    '''

    model = get_model(domain)
    
    model_input = tf.convert_to_tensor(get_observations_array(observations, max_plan_length))
    y_pred = model.predict(model_input)
    return y_pred



### GR Instance component methods

In [156]:
def get_score(prediction: np.ndarray, possible_goal: list) -> float:
    '''
    Returns the score for a possible goal.
    
    Args:
        prediction:
            An array that contains the model prediction.
        
        possible_goal:
            A list that contains the possible goal indexes.
        
    Returns:
        An float that represents the score of the possible goal.
    '''
    
    score=0
    
    for index in possible_goal:
        score += prediction[0][int(index)]
    return score

def get_scores(prediction: np.ndarray, possible_goals: list) -> np.ndarray:
    '''
    Returns the scores for all possible goals.
    
    Args:
        prediction:
            An array that contains the model prediction.
        
        possible_goals:
            A list of possible goals; each possible goal is represented as a
            list
        
    Returns:
        An array that contains the score of each of the possible goals.
    '''
    scores = np.zeros((len(possible_goals, )), dtype=float)
    for index, possible_goal in enumerate(possible_goals):
        scores[index] = get_score(prediction, possible_goal)
    return scores
        

def get_max(scores: np.ndarray) -> list:
    '''
    Returns a list with the index (or indexes) of the highest scores.
    
    Args:
        scores:
            An array that contains the scores as floats.
    
    Returns:
        A list thet contains the indexes of the highest score.
    '''
    max_element = -1
    index_max = list()
    for i in range(len(scores)):
        if scores[i] > max_element:
            max_element = scores[i]
            index_max = [i]
        elif scores[i] == max_element:
            index_max.append(i)

    return index_max
    
def get_result(scores: np.ndarray, correct_goal: int) -> bool:
    '''
    Computes if the goal recognition task is successfull.
    
    Args:
        scores:
            An array of floats that contains a score for 
            each possible goal
        correct_goal: 
            An integer that represents the index of the 
            correct goal
            
    Returns:
        True if the maximum score index corresponds to the 
        correct goal index, False otherwise.
    '''
    idx_max_list = get_max(scores)
    if len(idx_max_list) == 1:
        idx_max = idx_max_list[0]
    else:
        print(f'Algorithm chose randomly one of {len(idx_max_list)} equals candidates.')
        idx_max = idx_max_list[np.random.randint(0, len(idx_max_list))]
    if idx_max == correct_goal:
        return True
    else:
        return False
    
def get_correct_goal_idx(correct_goal: list, possible_goals: list) -> int:
    '''
    Conputes the correct goal index.
    
    Args:
        correct_goal:
            A list of strings that contains the correct goal
            fluents.
        possible_goals:
            A list of possible goals; each possible goal is represented as a
            list.
    
    Returns:
        The index of the correct goal in the possible goals list.
        None if the possible goal list does not contain the correct goal.
    '''
    
    for index, possible_goal in enumerate(possible_goals):
        possible_goal = np.sort(possible_goal)
        correct_goal = np.sort(correct_goal)
        if np.all(possible_goal == correct_goal):
            return index
    return None

### GRNet execution methods

In [157]:
def init_models(model_type: int, percentage: float)-> None:
    '''
    Loads in memory all the models.
    
    Args:
        model_type:
            an integer associated to the type
            of RNN model in use.
        
        percentage:
            a float that represents the model
            percentage to use. Use only with
            model_type = C.PERCENTAGE.
    
    Returns:
        None   
    '''
    
    model_file = get_domain_related(C.LOGISTICS, C.MODEL_FILE, model_type=model_type, percentage=percentage)
    C.MODEL_LOGISTICS =  load_model(join(C.MODELS_DIR, model_file), custom_objects=C.CUSTOM_OBJECTS)
    
    model_file = get_domain_related(C.SATELLITE, C.MODEL_FILE, model_type=model_type, percentage=percentage)
    C.MODEL_SATELLITE = load_model(join(C.MODELS_DIR, model_file), custom_objects=C.CUSTOM_OBJECTS)
    
    model_file = get_domain_related(C.ZENOTRAVEL, C.MODEL_FILE, model_type=model_type, percentage=percentage)
    C.MODEL_ZENOTRAVEL = load_model(join(C.MODELS_DIR, model_file), custom_objects=C.CUSTOM_OBJECTS)
    
    model_file = get_domain_related(C.DEPOTS, C.MODEL_FILE, model_type=model_type, percentage=percentage)
    C.MODEL_DEPOTS = load_model(join(C.MODELS_DIR, model_file), custom_objects=C.CUSTOM_OBJECTS)
    
    model_file = get_domain_related(C.DRIVERLOG, C.MODEL_FILE, model_type=model_type, percentage=percentage)
    C.MODEL_DRIVERLOG =  load_model(join(C.MODELS_DIR, model_file), custom_objects=C.CUSTOM_OBJECTS)
    
    model_file = get_domain_related(C.BLOCKSWORLD, C.MODEL_FILE, model_type=model_type, percentage=percentage)
    C.MODEL_BLOCKSWORLS =  load_model(join(C.MODELS_DIR, model_file), custom_objects=C.CUSTOM_OBJECTS)

In [158]:
def run_experiment(obs_file: str, 
            goals_dict_file: Union[str, None],
            actions_dict_file: Union[str, None],
            possible_goals_file: str, 
            correct_goal_file: str, 
            domain: Union[str, int], 
            verbose: int = 0) -> list:
    '''
    Run the goal recognition experiment

    Args:
        obs_file:
            Path of the file that contains the
            observations (plan)

        goals_dict_file:
            Path of the file that contains the
            goals dictionaries. If None it is
            retrieved from its default location.

        actions_dict_file:
            Path of the file that contains the
            actions dictionaries. If None it is
            retrieved from its default location.

        possible_goals_file:
            Path of the file that contains the
            possible goals.

        correct_goal_file:
            Path of the file that contains the
            correct goal.

        domain:
            String that contains the name of the
            domain or integer that corresponds to
            a domain.

        verbose:
            Integer that corresponds to how much
            information is printed. 0 = no info,
            2 = max info

    Returns:
         A list that contains the result, the correct
         goal index and the predicted goal index.
    '''

    domain = parse_domain(domain)
    if goals_dict_file is None:
        goals_dict_file = join(get_domain_related(domain, C.DICTIONARIES_DICT), 'dizionario_goal')
    goals_dict = load_file(goals_dict_file, binary=True, use_pickle=True)
    if actions_dict_file is None:
        actions_dict_file = join(get_domain_related(domain, C.DICTIONARIES_DICT), 'dizionario')
    actions_dict = load_file(actions_dict_file, binary=True, use_pickle=True)
    observations = parse_file(obs_file, C.OBSERVATIONS, actions_dict)
    
    if verbose > 1:
        print('Observed actions:\n')
        for o in observations:
            print(o)
    possible_goals = parse_file(possible_goals_file, C.POSSIBLE_GOALS, goals_dict)
    
    max_plan_length = get_domain_related(domain, C.MAX_PLAN_LENGTH)
    predictions = get_predictions(observations, max_plan_length, domain)
    scores = get_scores(predictions, possible_goals)
    if verbose > 0:
        for index, goal in enumerate(possible_goals):
            print(f'{index} - {goal} : {scores[index]}')
    
    correct_goal = parse_file(correct_goal_file, C.CORRECT_GOAL, goals_dict) 
    correct_goal_idx = get_correct_goal_idx(correct_goal, possible_goals)
    result = get_result(scores, correct_goal_idx)
    if verbose > 0:
        print(f'Predicted goal is {get_max(scores)[0]}')
        print(f'Correct goal is {correct_goal_idx} - {correct_goal}')
    return [result, correct_goal_idx, get_max(scores)[0]]



## GRNet execution

Do not change these values

In [159]:
model_type=C.SMALL 
percentage=0

init_models(model_type=model_type, percentage=percentage)

Change these values to fit your execution

In [ ]:
domain = C.BLOCKSWORLD
domain_dir = f'../testsets/TS_PerGen_noisy/blocksworld-noisy'
temp_dir = f'./files_temp_dir'
verbose = 0

In [ ]:
def process_domain(
    domain: int,
    domain_dir: str,
    temp_dir: str = './files_temp_dir',
    perc_list: List[float] = [0.1, 0.3, 0.5, 0.7, 1],
    verbose: int = 0
) -> Dict[str, pd.DataFrame]:
    '''
    Process a domain for different observation percentages.
    
    Args:
        domain:
            An integer associated to a specific domain.
        
        domain_dir:
            A string that contains the path to the
            directory that contains the testset.
        
        temp_dir:
            A string that contains the path to a temporary
            directory to extract files to.
        
        perc_list:
            Optional. A list of floats that contains the
            observation percentages to process.
        
        verbose:
            Optional. An integer that corresponds to how much
            information is printed. 0 = no info, 1 = max info.
    
    Returns:
        A dictionary where each key is a string representing
        the observation percentage and each value is a DataFrame
        that contains the results for that observation percentage.
    '''
    results = {}
    times = list()
    for perc in perc_list:
        plans_dir = f'{join(domain_dir, str(int(perc*100)))}'
        files = os.listdir(plans_dir)
        total=0
        correct=0
        rows = []

        domain_name = os.path.basename(domain_dir)
        for j, f in enumerate(tqdm(files, desc=f'[{domain_name}][{int(perc*100)}%] Processing plans', unit='plans')): 
            obs_dir = join(plans_dir, f)
            if verbose:
                print(f"File: {f}")
            if f.endswith('.zip'):
                unzip_file(join(plans_dir,f), temp_dir)
                obs_dir = temp_dir
            elif f.endswith('.bz2'):
                unpack_bz2(join(plans_dir,f), temp_dir)
                obs_dir = temp_dir
            start_time = time.time()
            result = run_experiment(obs_file=join(obs_dir, 'obs.dat'),
                                    goals_dict_file=None,
                                    actions_dict_file=None,
                                    possible_goals_file=join(obs_dir, 'hyps.dat'),
                                    correct_goal_file=join(obs_dir, 'real_hyp.dat'),
                                    domain=domain, 
                                    verbose=verbose)
            exec_time = time.time()-start_time
            if result[0]:
                correct+=1
            total +=1
            times.append(exec_time)
            if verbose:
                print(f'Execution time: {exec_time} seconds')
            rows.append({
                'problem': f,
                'correct_goal_idx': result[1],
                'predicted_goal_idx': result[2]
            })

        results_df = pd.DataFrame(rows)
        results[f"{int(perc*100)}"] = results_df
    return results

In [ ]:
def process_noisy_domain(
    domain: int,
    domain_dir: str,
    temp_dir: str = './files_temp_dir',
    perc_list: List[float] = [0.1, 0.3, 0.5, 0.7, 1],
    noise_perc_list: List[float] = [0.05, 0.1, 0.2, 0.3],
    verbose: int = 0
) -> Dict[str, Dict[str, pd.DataFrame]]:
    '''
    Process a noisy domain for different observation percentages and noise levels.
    
    Args:
        domain:
            An integer associated to a specific domain.
        
        domain_dir:
            A string that contains the path to the
            directory that contains the testset. The inner
            directories (the ones named after noise levels)
            must have the noise level as a suffix (for example, 
            '_5' for 5% noise, '_10' for 10% noise, etc.).
        
        temp_dir:
            A string that contains the path to a temporary
            directory to extract files to.
        
        perc_list:
            Optional. A list of floats that contains the
            observation percentages to process.
        
        noise_perc_list:
            Optional. A list of floats that contains the
            noise levels to process.
        
        verbose:
            Optional. An integer that corresponds to how much
            information is printed. 0 = no info, 1 = max info.
    
    Returns:
        A dictionary where each key is a string representing
        the observation percentage and each value is another
        dictionary. In this second dictionary, each key is a
        string representing the noise level and each value is
        a DataFrame that contains the results for that noise
        level.
    '''
    results = {}
    for noise in noise_perc_list:
        noise_dir = f'{domain_dir}_{int(noise*100)}'
        print(f'=== Processing noise level: {int(noise*100)}% ===')
        results[f"{int(noise*100)}"] = process_domain(
            domain,
            noise_dir,
            temp_dir=temp_dir,
            perc_list=perc_list,
            verbose=verbose
        )
    
    return results


In [168]:
results = process_domain(
    domain=domain,
    domain_dir='../testsets/TS_PerGen/blocksworld',
)

[blocksworld][10%] Processing plans:   0%|          | 0/754 [00:00<?, ?plans/s]/tmp/ipykernel_19066/1374631018.py:112: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if np.all(possible_goal == correct_goal):
[blocksworld][100%] Processing plans: 100%|██████████| 754/754 [00:15<00:00, 47.53plans/s]


In [169]:
results["100"].head()

,problem,correct_goal_idx,predicted_goal_idx
0,blocksworld_p000684_hyp=hyp-13_100.zip,13,13
1,blocksworld_p001170_hyp=hyp-3_100.zip,3,1
2,blocksworld_p000207_hyp=hyp-16_100.zip,16,16
3,blocksworld_p001396_hyp=hyp-12_100.zip,12,12
4,blocksworld_p002053_hyp=hyp-6_100.zip,6,6


In [ ]:
results = process_noisy_domain(
    domain=domain,
    domain_dir='../testsets/TS_PerGen_noisy/blocksworld-noisy'
)

=== Processing noise level: 5% ===


[blocksworld-noisy_5][10%] Processing plans:   0%|          | 0/754 [00:00<?, ?plans/s]/tmp/ipykernel_19066/1374631018.py:112: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if np.all(possible_goal == correct_goal):
[blocksworld-noisy_5][100%] Processing plans: 100%|██████████| 754/754 [00:13<00:00, 56.94plans/s]


=== Processing noise level: 10% ===


[blocksworld-noisy_10][100%] Processing plans: 100%|██████████| 754/754 [00:13<00:00, 56.94plans/s]


=== Processing noise level: 20% ===


[blocksworld-noisy_20][100%] Processing plans: 100%|██████████| 754/754 [00:13<00:00, 55.76plans/s]


=== Processing noise level: 30% ===


[blocksworld-noisy_30][100%] Processing plans: 100%|██████████| 754/754 [00:13<00:00, 56.38plans/s]


In [167]:
results["5"]["100"].head()

,problem,correct_goal_idx,predicted_goal_idx
0,blocksworld_p001053_hyp=3_100,3,5
1,blocksworld_p003410_hyp=15_100,15,15
2,blocksworld_p000641_hyp=17_100,17,17
3,blocksworld_p001046_hyp=16_100,16,16
4,blocksworld_p001479_hyp=19_100,19,19


## GRNet Evaluation

In [ ]:
# Extract final results for observation percentage
obs_results = results["10"]
result_report = classification_report(obs_results['correct_goal_idx'], obs_results['predicted_goal_idx'], zero_division=0)
accuracy = accuracy_score(obs_results['correct_goal_idx'], obs_results['predicted_goal_idx'])
print('Classification Report:\n')
print(result_report)
print(f'Accuracy: {accuracy}')

Classification Report:

              precision    recall  f1-score   support

           0       0.17      0.21      0.19        39
           1       0.21      0.25      0.23        36
           2       0.16      0.13      0.14        45
           3       0.19      0.23      0.21        43
           4       0.09      0.07      0.08        29
           5       0.19      0.15      0.17        39
           6       0.22      0.16      0.19        37
           7       0.09      0.10      0.09        40
           8       0.23      0.17      0.20        41
           9       0.30      0.15      0.20        40
          10       0.11      0.16      0.13        25
          11       0.24      0.19      0.21        37
          12       0.16      0.19      0.18        36
          13       0.26      0.31      0.28        26
          14       0.25      0.34      0.29        38
          15       0.21      0.19      0.20        42
          16       0.25      0.26      0.26        38
   

In [ ]:
# Extract final results for observation percentage
obs_results = results["30"]
result_report = classification_report(obs_results['correct_goal_idx'], obs_results['predicted_goal_idx'], zero_division=0)
accuracy = accuracy_score(obs_results['correct_goal_idx'], obs_results['predicted_goal_idx'])
print('Classification Report:\n')
print(result_report)
print(f'Accuracy: {accuracy}')

Classification Report:

              precision    recall  f1-score   support

           0       0.48      0.38      0.43        39
           1       0.48      0.61      0.54        36
           2       0.55      0.38      0.45        45
           3       0.49      0.44      0.46        43
           4       0.44      0.52      0.48        29
           5       0.39      0.38      0.39        39
           6       0.48      0.57      0.52        37
           7       0.49      0.65      0.56        40
           8       0.66      0.46      0.54        41
           9       0.42      0.42      0.42        40
          10       0.31      0.32      0.31        25
          11       0.52      0.43      0.47        37
          12       0.65      0.56      0.60        36
          13       0.41      0.42      0.42        26
          14       0.38      0.55      0.45        38
          15       0.37      0.36      0.36        42
          16       0.49      0.50      0.49        38
   

In [ ]:
# Extract final results for observation percentage
obs_results = results["50"]
result_report = classification_report(obs_results['correct_goal_idx'], obs_results['predicted_goal_idx'], zero_division=0)
accuracy = accuracy_score(obs_results['correct_goal_idx'], obs_results['predicted_goal_idx'])
print('Classification Report:\n')
print(result_report)
print(f'Accuracy: {accuracy}')

Classification Report:

              precision    recall  f1-score   support

           0       0.61      0.56      0.59        39
           1       0.60      0.78      0.67        36
           2       0.68      0.47      0.55        45
           3       0.69      0.67      0.68        43
           4       0.56      0.79      0.66        29
           5       0.74      0.72      0.73        39
           6       0.68      0.81      0.74        37
           7       0.48      0.78      0.60        40
           8       0.75      0.66      0.70        41
           9       0.73      0.55      0.63        40
          10       0.45      0.60      0.52        25
          11       0.76      0.59      0.67        37
          12       0.75      0.50      0.60        36
          13       0.64      0.69      0.67        26
          14       0.67      0.79      0.72        38
          15       0.74      0.62      0.68        42
          16       0.61      0.58      0.59        38
   

In [ ]:
# Extract final results for observation percentage
obs_results = results["70"]
result_report = classification_report(obs_results['correct_goal_idx'], obs_results['predicted_goal_idx'], zero_division=0)
accuracy = accuracy_score(obs_results['correct_goal_idx'], obs_results['predicted_goal_idx'])
print('Classification Report:\n')
print(result_report)
print(f'Accuracy: {accuracy}')

Classification Report:

              precision    recall  f1-score   support

           0       0.81      0.90      0.85        39
           1       0.69      0.75      0.72        36
           2       0.90      0.60      0.72        45
           3       0.82      0.84      0.83        43
           4       0.81      0.90      0.85        29
           5       0.87      0.85      0.86        39
           6       0.88      0.95      0.91        37
           7       0.77      0.85      0.81        40
           8       0.92      0.83      0.87        41
           9       0.94      0.82      0.88        40
          10       0.74      0.92      0.82        25
          11       0.91      0.86      0.89        37
          12       0.94      0.83      0.88        36
          13       0.92      0.88      0.90        26
          14       0.92      0.95      0.94        38
          15       0.90      0.86      0.88        42
          16       0.80      0.92      0.85        38
   

In [ ]:
# Extract final results for observation percentage
obs_results = results["100"]
result_report = classification_report(obs_results['correct_goal_idx'], obs_results['predicted_goal_idx'], zero_division=0)
accuracy = accuracy_score(obs_results['correct_goal_idx'], obs_results['predicted_goal_idx'])
print('Classification Report:\n')
print(result_report)
print(f'Accuracy: {accuracy}')

Classification Report:

              precision    recall  f1-score   support

           0       0.83      0.97      0.89        39
           1       0.81      0.83      0.82        36
           2       0.97      0.69      0.81        45
           3       1.00      0.86      0.92        43
           4       0.76      1.00      0.87        29
           5       0.95      0.90      0.92        39
           6       1.00      1.00      1.00        37
           7       0.93      0.97      0.95        40
           8       0.95      1.00      0.98        41
           9       1.00      0.95      0.97        40
          10       0.92      0.88      0.90        25
          11       0.92      0.97      0.95        37
          12       0.92      0.97      0.95        36
          13       1.00      1.00      1.00        26
          14       1.00      0.97      0.99        38
          15       0.97      0.93      0.95        42
          16       0.95      0.97      0.96        38
   